In [1]:
# Import required packages
import numpy as np
import galois
import random 
from py_ecc.bn128 import G1, G2, multiply, add, neg, pairing, curve_order, Z1
from functools import reduce

In [2]:
"""
The task is to evaluate out <== 3xˆ3 - 5xˆ2yˆ2 + 7xyˆ2 - 21y + 11

- Constraints

v1 = xx
v2 = yy
v3 = 3xv1
v4 = 5v1v2
out - 11 - v3 + v4 + 21y = 7xv2    -> as addition is 'free', we move them to the output side

witness = [1, out, x, y, v1, v2, v3, v4]
"""
# Initialize a finite field 
p = curve_order
Fp = galois.GF(p)
print("Galois field: ", Fp)

# Matrices L, R, and O are drawn from the above constraints 
L = Fp([[0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 3, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 5, 0, 0, 0],
        [0, 0, 7, 0, 0, 0, 0, 0]])

R = Fp([[0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0]])

O = Fp([[0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [Fp(p - 11), 1, 0, 21, 0, 0, Fp(p - 1), 1]])    # in GF(p), -1 = p - 1, and -11 = p - 11

# Compute the witness
x = Fp(random.randint(1, p))
y = Fp(random.randint(1, p))
v1 = x * x
v2 = y * y
v3 = 3 * x * v1
v4 = 5 * v1 * v2
out = v3 - v4 - 21 * y + 7 * x * v2 + Fp(11)

witness = Fp([1, out, x, y, v1, v2, v3, v4])  
print("Witness vector: ", witness)

Galois field:  <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
Witness vector:  [                                                                            1
  7775573850847386489551847623684343099244235897988069798249451110407634766666
 14471461404627795793695734789806779032792678691387592207325835355631177143759
 21764905880703304717573902884442041066447275857622489041390664233453269174933
  3801390595967506878919913865618962614827377806042337455847183389033776208028
 16155782066385206988511716264749044543418615109453637250333055830789010103448
  6763845798566572611097455071601620753071984816235116870222183477835485323677
  9481753646908141538194129245715740924427580553645104182069092246981238992593]


In [3]:
# As O*witness = (L*witness) * (R*witness)
print("L*w = ", np.dot(L, witness))
print("R*w = ", np.dot(R, witness))
print("O*w = ", np.dot(O, witness))
print("L*w * Rw = ", np.dot(L, witness) * np.dot(R, witness))
assert all(np.dot(O, witness) == np.dot(L, witness) * np.dot(R, witness))

L*w =  [14471461404627795793695734789806779032792678691387592207325835355631177143759
 21764905880703304717573902884442041066447275857622489041390664233453269174933
 21526141342044112158840798624163062009829671673746742278279301880317722935660
 19006952979837534394599569328094813074136889030211687279235916945168881040140
 13747258345037469666884520547618352875355293238049008076488030743115006023845]
R*w =  [14471461404627795793695734789806779032792678691387592207325835355631177143759
 21764905880703304717573902884442041066447275857622489041390664233453269174933
  3801390595967506878919913865618962614827377806042337455847183389033776208028
 16155782066385206988511716264749044543418615109453637250333055830789010103448
 16155782066385206988511716264749044543418615109453637250333055830789010103448]
O*w =  [ 3801390595967506878919913865618962614827377806042337455847183389033776208028
 16155782066385206988511716264749044543418615109453637250333055830789010103448
  676384579856657261109745507

In [4]:
"""
Transform the r1cs constraints to Quadratic Arithmetic Program
    --> move from vector space (including L, R, O and witness vector) to polynomial space
    --> Results are 5 polynomials: lists of u_i(x), v_i(x), w_i(x) (for i in range(len(witness)), U(x), V(x), W(x), t(x) and h(x)
"""

# Function returns a list of polynomials of degree d, interpolated from columns of matrix M, 
# and a sum of inner product of polys with the witness vector w
def transform_matrix2poly(M, w, d):
    poly_list = []
    # Interpolate the matrix M to a polynomial, d: degree of the polynomial
    xs = Fp(np.array([i + 1 for i in range(d + 1)]))    
    poly_sum = galois.Poly([0] * (d + 1), field=Fp)
    for i in range(len(w)):
        poly = galois.lagrange_poly(xs, M[:, i])
        poly_list.append(poly)
        poly_sum += poly * w[i]
    return poly_sum, poly_list

n = len(L[:, 0])        # number of rows, equivalent to the number of constraints
d = n - 1               # degree polynomials interpolated from matrices' columns
m = len(L[0, :])        # number of columns = len(witness)
print("Number of rows of matrices: ", n);
print("Number of columns of matrices (or elements of witness): ", m)

U_poly, u_list = transform_matrix2poly(L, witness, d)   # u_list = list of u_i(x), U(x) = sum(w_i u_i(x))
V_poly, v_list = transform_matrix2poly(R, witness, d)   # u_list = list of u_i(x), V(x) = sum(w_i v_i(x))
W_poly, w_list = transform_matrix2poly(O, witness, d)   # u_list = list of u_i(x), W(x) = sum(w_i w_i(x))

print("U(x) = sum(w_i u_i(x)): ", U_poly)
print("V(x) = sum(w_i v_i(x)): ", V_poly)
print("W(x) = sum(w_i w_i(x)): ", W_poly)

t_poly = galois.Poly.Roots(np.array([i + 1 for i in range(d + 1)]), field=Fp)
UV_poly = U_poly * V_poly
h_poly = (UV_poly - W_poly) // t_poly
remainder = (UV_poly - W_poly) % t_poly
# Check if the remainder is equal to zero
print("The remainder polynomial: ", remainder)
assert remainder == 0, "The remainder polynomial is not equal to 0!"

"""
Verify the equality of polynomials
    --> U(x)*V(x) = W(x) + t(x)*h(x)
"""

assert UV_poly == W_poly + t_poly * h_poly, "QAP not equal!"
print("Product of two polys U(x) * V(x): ", UV_poly)
print("W(x) + t(x)*h(x): ", W_poly + t_poly * h_poly)

Number of rows of matrices:  5
Number of columns of matrices (or elements of witness):  8
U(x) = sum(w_i u_i(x)):  7058086472270032655954266841429762004715677400996192642523707756615432828592x^4 + 17847404296521327042293479119918975508902854073444100176429767288306296132248x^3 + 8188586676584086106403295628258189978957759264991724719666363258070998047318x^2 + 14583471748694774136256793692068734526306552647200543926580201975143234482008x + 10570397954236126297280710998645667191006564105587099429522203450646832644827
V(x) = sum(w_i v_i(x)):  7762499691348940630024488847493743040701697248924966478913490237047468242703x^4 + 4598290411968978559039455431155403365725180607604866323132055419514064535094x^3 + 17434078390208624569718633739453497063775428127123510831095604841229911922417x^2 + 3359866897986152851296397853096277468661869401500111988435110541004157338273x + 3204968884954374405863164663865133182476867706650170929447778503411383600889
W(x) = sum(w_i w_i(x)):  6417690014602062051644350

In [5]:
"""
Trusted setup: The groth16's trusted setup consists of two phases: 
- Phase 1: Generate a random value 'tau' and its powers that are independent from all circuits
- Phase 2: Generate proving and verification keys for a specific circuit
"""
def generate_powers_of_tau(tau, degree, point):
    return [multiply(point, int(tau ** i)) for i in range(degree + 1)]

def inner_product(ec_points, coeffs):
    # Check if number of ec_points equal to the one of coeffs
    assert len(coeffs) == len(ec_points), "Check failed, number of points is different from number of coeffs!"

    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)

# Phase 1 
tau = random.randint(1, p)
print("tau MUST be secret, trusted server will detroyed it after generating its powers, tau = ", tau) 
# tau^0*G1, tau*G1, tauˆ2*G1, ..., tauˆd*G1
powers_of_tau_for_G1 = generate_powers_of_tau(tau, d, G1)
# tau^0*G2, tau*G2, tauˆ2*G2, ..., tauˆd*G2
powers_of_tau_for_G2 = generate_powers_of_tau(tau, d, G2)
t_tau = t_poly(tau)
powers_of_tau_for_ht = [multiply(powers_of_tau_for_G1[i], int(t_tau)) for i in range(d)]


# Phase 2
# Prover generates proof
A1 = inner_product(powers_of_tau_for_G1, U_poly.coeffs[::-1])
B1 = inner_product(powers_of_tau_for_G2, V_poly.coeffs[::-1])
Cw = inner_product(powers_of_tau_for_G1, W_poly.coeffs[::-1])
evaluate_ht_on_G1_1 = inner_product(powers_of_tau_for_ht, h_poly.coeffs[::-1])
C1 = add(Cw, evaluate_ht_on_G1_1)

# Verification
if pairing(B1, A1) == pairing(G2, C1): 
    print("Passed check #1. The proof is correct with unmodified A, B, C!")
else:
    print("Proof incorrect!") 

tau MUST be secret, trusted server will detroyed it after generating its powers, tau =  8390646662653619683060454939356964359922790597773192303923697854736134504989
Passed check #1. The proof is correct with unmodified A, B, C!


In [6]:
print(f"[A]G1 = {A1}")
print(f"[B]G2 = {B1}")
print(f"[Cw]G1 = {Cw}")
print(f"[C]G1 = {C1}")
print(f"[ht]G1 = {evaluate_ht_on_G1_1}")

[A]G1 = (16120848897776662565189918637869612241944596851233621165148962020008574339657, 20368152155216447751286490789125917743954209697133846476226101156604680297865)
[B]G2 = ((12337485224473930372021896426428773925111409809065999597116641910021348549769, 18396857119319705351974156884655369455994305679262406808614099290027983831435), (1926255116067745999450455607221097615344354859229309160898818454762378019534, 17463921644929315669468815195152532136079140704989333799909020788365645824508))
[Cw]G1 = (12479547032439429096642142384328059377825117313650141237856967610295018257573, 8188104842207638912994203776269997648225551171665461697428719774400763494267)
[C]G1 = (17242622110903129719965874298823162898139034282899039552990805162607746102660, 19019861627253942181327740649051914260579482979327559120186680351068168078069)
[ht]G1 = (9261412047496097106970595773857523268596062812058581830392663388965223019611, 21753616733993413565650919974044050030195817314300500002745454820944926128937)


In [7]:
"""
To prevent a dishonest prover from inventing a proof (A, B, C), a secret shifting technique is introduced 
"""
# Trusted server generates alpha and beta
alpha = random.randint(1, p)
beta = random.randint(1, p)
alpha_G1 = multiply(G1, int(alpha))     # alpha*G1
beta_G2 = multiply(G2, int(beta))       # beta*G2

def generate_powers_of_tau_for_C(powers_of_tau, d, m, L_mat, R_mat, O_mat, a, b):
    ret = []
    # require degree + 1 points to interpolate a polynomial of degree d
    xs = Fp(np.array([i + 1 for i in range(d + 1)]))
    # Each i-th col of matrices L, R, and W will be converted to a polynomial U_i(x), V_i(x) and W_i(x)
    for i in range(m):
        # U_i(x) = interpolate the i-th column of matrix L
        poly_Ui = galois.lagrange_poly(xs, L_mat[:, i])
        # Perform a random shift by multiplying the poly with a random factor beta
        beta_Ui = poly_Ui * b        # multiply U with beta
        # V_i(x) = interpolate the i-th column of matrix R
        poly_Vi = galois.lagrange_poly(xs, R_mat[:, i])
        # Perform a random shift by multiplying the poly with a random factor alpha
        alpha_Vi = poly_Vi * a
        # W_i(x) = interpolate the i-th column of matrix W
        poly_Wi = galois.lagrange_poly(xs, O_mat[:, i])
        # Sum of polys
        sum_poly = beta_Ui + alpha_Vi + poly_Wi
        
        # Evaluate the sum polynomial at tau, then multiply with point G1
        #       = (beta*U_i + alpha*V_i + W_i)(tau)*G1
        tau_for_C = inner_product(powers_of_tau, sum_poly.coeffs[::-1])
        ret.append(tau_for_C)
    return ret
    
taus_for_C = generate_powers_of_tau_for_C(powers_of_tau_for_G1, d, m, L, R, O, alpha, beta)

# Prover generates proof
A2 = add(alpha_G1, A1)        # random shift for A, [alpha + U(tau)]*G1
B2 = add(beta_G2, B1)          # random shift for B, [beta + V(tau)]*G2
C_shift = inner_product(taus_for_C, witness)
C2 = add(C_shift, evaluate_ht_on_G1_1)
    
# Verification
if pairing(B2, A2) == pairing(beta_G2, alpha_G1) * pairing(G2, C2):
    print("Pass test #2, proof is correct after introducing alpha and beta!")
else:
    print("Proof incorrect!") 

Pass test #2, proof is correct after introducing alpha and beta!


In [8]:
print(f"[alpha_A]G1 = {A2}")
print(f"[beta_B]G2 = {B2}")
print(f"Cw with secret shift= {C_shift}")
print(f"C with secret shift= {C2}")

[alpha_A]G1 = (419450152367147339485068066320756531721511335522937077896755470145337949210, 20301763597314082410784828770189154702250893623668970152805125095437495250675)
[beta_B]G2 = ((6418920467410782174782719814429438656569856472015671133497013396353568480409, 18895244054113451962927896519228916777785991650446067259997782296828751706384), (1416914047573087825279887130768894487478152771985891058661583494291027671108, 948948837795359801432286827375213210714157580771625808222805401047834623799))
Cw with secret shift= (11852052163167038940000453345511886472713957682561462412800253142343458940643, 10619344730561813181604785145702083042689750791299368075481063297838848378577)
C with secret shift= (19379914920844106750386814899197487462704153473286450037930848612096923825876, 3273118210855761670488524772279895736400453728061099751424355232453358034784)


In [9]:
"""
    Separate public and private inputs with gamma and delta
"""
def generate_powers_of_tau_for_inputs(powers_of_tau, d, m, L_mat, R_mat, O_mat, a, b, l, g_inv, d_inv):
    taus_for_pub_inputs = []
    taus_for_priv_inputs = []
    xs = Fp(np.array([i + 1 for i in range(d + 1)]))
    for i in range(m):
        poly_Ui = galois.lagrange_poly(xs, L_mat[:, i])
        beta_Ui = poly_Ui * b        # multiply U with beta
        poly_Vi = galois.lagrange_poly(xs, R_mat[:, i])
        alpha_Vi = poly_Vi * a
        poly_Wi = galois.lagrange_poly(xs, O_mat[:, i])
        sum_poly = beta_Ui + alpha_Vi + poly_Wi
        
        if i < l:
            taus_for_pub_inputs.append(inner_product(powers_of_tau, (sum_poly.coeffs[::-1])*g_inv))
        else:
            taus_for_priv_inputs.append(inner_product(powers_of_tau, (sum_poly.coeffs[::-1])*d_inv))
    return taus_for_pub_inputs, taus_for_priv_inputs
    
ell = 2  # number of public inputs. In this example, public inputs are 1 and out
taus_for_public_inputs, taus_for_private_inputs = generate_powers_of_tau_for_inputs(powers_of_tau_for_G1, d, m, L, R, O, alpha, beta, ell, 1, 1)

# Powers of taus for public and private inputs at this step should be same as powers of taus for C before separating inputs
assert taus_for_C == taus_for_public_inputs + taus_for_private_inputs

C_public = inner_product(taus_for_public_inputs, witness[:ell])
C_private_taus = inner_product(taus_for_private_inputs, witness[ell:])
C_private = add(C_private_taus, evaluate_ht_on_G1_1)

# Likewise values of C should be the same as before separating
assert C_shift == add(C_public, C_private_taus)
assert C2 == add(C_public, C_private)

# Check #2
if pairing(B2, A2) == pairing(beta_G2, alpha_G1) * pairing(G2, C_public) * pairing(G2, C_private):
    print("Pass test #2, proof is correct after separating public and private inputs!")
else:
    print("Proof incorrect!")

Pass test #2, proof is correct after separating public and private inputs!


In [10]:
"""
    Introducing gamma and delta to prevent forgeries with public inputs 
"""
gamma = random.randint(1, p)
gamma_G2 = multiply(G2, int(gamma))  # gamma*G2
gamma_inv = Fp(1) / Fp(gamma)
delta = random.randint(1, p)
delta_G1 = multiply(G1, int(delta))  # delta*G1
delta_G2 = multiply(G2, int(delta))  # delta*G2
delta_inv = Fp(1) / Fp(delta)

t_tau_delta = Fp(t_poly(tau) * delta_inv)
powers_of_tau_for_ht2 = [multiply(powers_of_tau_for_G1[i], int(t_tau_delta)) for i in range(d)]
evaluate_ht_on_G1_2 = inner_product(powers_of_tau_for_ht2, h_poly.coeffs[::-1])
taus_for_public_inputs2, taus_for_private_inputs2 = generate_powers_of_tau_for_inputs(powers_of_tau_for_G1, d, m, L, R, O, alpha, beta, ell, gamma_inv, delta_inv)

C_public2 = inner_product(taus_for_public_inputs2, witness[:ell])
C_private_taus2 = inner_product(taus_for_private_inputs2, witness[ell:])
C_private2 = add(C_private_taus2, evaluate_ht_on_G1_2)

# Check #3
if pairing(B2, A2) == pairing(beta_G2, alpha_G1) * pairing(delta_G2, C_private2) * pairing(gamma_G2, C_public2):
    print("Pass test #3, proof is correct after introducing gamma and delta to prevent forgeries with public inputs!")
else:
    print("Proof incorrect!")
        

Pass test #3, proof is correct after introducing gamma and delta to prevent forgeries with public inputs!


In [11]:
"""
    Enforcing true Zero-Knowledge by introducing two random values r and s
    Prover samples random value r, and s to prevent verifier from guessing witness values
"""
r = random.randint(1, p)
s = random.randint(1, p)
r_delta_G1 = multiply(delta_G1, int(r))
s_delta_G1 = multiply(delta_G1, int(s))
s_delta_G2 = multiply(delta_G2, int(s))
rs_delta_G1 = multiply(r_delta_G1, int(s))

# Update A, B
A3 = add(A2, r_delta_G1);
beta_G1 = multiply(G1, int(beta))       # beta*G1
B_G1 = inner_product(powers_of_tau_for_G1, V_poly.coeffs[::-1])
B_beta_G1 = add(B_G1, beta_G1)
B3_G1 = add(B_beta_G1, s_delta_G1);
B3 = add(B2, s_delta_G2);
sA = multiply(A3, int(s))
rB1 = multiply(B3_G1, int(r))


"""
    Compute C = (W(x) + beta*U(x) + alpha*V(x))*w + h(x)t(x)  + sA + rB1 + rs*delta*G1
"""
#C_private_inputs = inner_product(taus_for_private_inputs, witness[ell:])
#C_ht = add(C_private_inputs, evaluate_ht_on_G1)
C_ht_sa = add(C_private2, sA);
C_ht_sa_rb = add(C_ht_sa, rB1);
C_private3 = add(C_ht_sa_rb, neg(rs_delta_G1))
C_public3 = C_public2

"""
Verifying: verifier obtains a proof from prover, consisting of three elliptic curve points:
    - [A, B, C]
    and two points from trusted server:
    - alpha_G1
    - beta_G2
        
    Accept the proof if the below equation is true. Otherwise, reject
"""
    
# Final check
if pairing(B3, A3) == pairing(beta_G2, alpha_G1) * pairing(delta_G2, C_private3) * pairing(gamma_G2, C_public3):
    print("Pass final test after adding two random values (r, s) to ensure the zero-knowledge!")
else:
    print("Failed final test")    

Pass final test after adding two random values (r, s) to ensure the zero-knowledge!
